In [1]:
import time
import numpy as np
import pandas as pd
from fastparquet import ParquetFile, write
import matplotlib.pyplot as plt
import timeit
import dask.dataframe as dd
import holoviews as hv
import datashader as ds
import gc
from os.path import join, dirname
import pyarrow 
import pyarrow.parquet as pq
# PLOT USING HOLOVIEWS DASK AND DATASHADER
import hvplot.pandas
import hvplot.dask
import hvplot as hv
from bokeh.models import HoverTool
from pdb import set_trace
from datetime import datetime, timedelta
import csv
import dateutil.relativedelta
from parse import *
from numpy.core.numerictypes import *

%matplotlib inline

In [2]:
import findspark
findspark.init('/usr/local/spark/spark-2.3.2-bin-hadoop2.7')

import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StringType, IntegerType
from pyspark.sql import *
from pyspark.sql.window import Window

spark = SparkSession \
    .builder \
    .appName("Poolminers") \
    .getOrCreate()

sc=SparkContext.getOrCreate(spark)

In [27]:
# LOAD BLOCK SAMPLE
df = spark.read.parquet('../data5/block.parquet').select('block_timestamp','transaction_list',
                      'miner_address','block_number')
df = df.where(col('block_number')<=1289357)
df.select('transaction_list').show(5,False)
# TRANSACTION LIST ORDER
# Hash,from,to,Value,timestamp,block_number

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|transaction_list                                                                                                                                                                                                                                  |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[]                                                                                                                                                                                                                                                |
|[["f44b6a843d8a8c96

In [28]:
# UTILS
def explode_block(df1,col):
    # remove the outside double brackets
    df1 = df1.withColumn(col,regexp_replace('transaction_list', '(\[\[)','['))
    df1 = df1.withColumn(col,regexp_replace('transaction_list', '(\]\])',']'))

    ## explode the list the first time
    df1 = df1.withColumn(col,explode(split(f.col(col),'\],\[') ))
    # extract the values in the list into columns
    #df1 = df1.withColumn(col,regexp_replace('transaction_list', '(\[|\]|")', ''))
    #df1 = df1.withColumn(col, df1[col].substr(0, 64))
    return df1
df = explode_block(df,'transaction_list')
df.select('transaction_list').show(20, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|transaction_list                                                                                                                                                                                                                                |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[]                                                                                                                                                                                                                                              |
|["f44b6a843d8a8c9685a51155c

In [42]:
# convert list items to columns
def list_to_cols(x):
    if x in ['[]','']:
        pass
    else:
        return x[0]
df = list_to_cols(df,'transaction_list').show(20,False)

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.